# NLP

- Networks do not understand raw text so all text has to be encoded.
- Then it needs to be one-hot encoded

# Libraries

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Data

## Read data

In [2]:
with open('../Data/shakespeare.txt','r', encoding='utf8') as f:
    text = f.read()

In [3]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

## Encoding text

We will create a set of all characters, assign an id to each character, and build two dictionaries one with id > text, another with text > id

In [4]:
# Create a set of all unique characters in the text
all_characters = set(text)
len(all_characters)

84

In [5]:
# Create a decoder that reads the ID and returns the character.
# Assign an ID to each character and save it in a dictionary
decoder = dict(enumerate(all_characters))
decoder

{0: '<',
 1: 'z',
 2: 't',
 3: 'b',
 4: 'W',
 5: '`',
 6: 'e',
 7: '(',
 8: 'v',
 9: 'o',
 10: '}',
 11: 'P',
 12: 'n',
 13: '2',
 14: 'l',
 15: 'N',
 16: '4',
 17: 'q',
 18: '9',
 19: 'k',
 20: 'g',
 21: 'E',
 22: 'H',
 23: 'F',
 24: '1',
 25: '6',
 26: 'u',
 27: 'X',
 28: 'Q',
 29: '0',
 30: '>',
 31: 'S',
 32: '-',
 33: 'D',
 34: '\n',
 35: ';',
 36: "'",
 37: '5',
 38: 'U',
 39: 'V',
 40: ' ',
 41: 'I',
 42: '.',
 43: 'Y',
 44: 'x',
 45: ',',
 46: 'j',
 47: 'K',
 48: '&',
 49: 'C',
 50: 'A',
 51: 'm',
 52: 'h',
 53: 'O',
 54: 'i',
 55: 'Z',
 56: '3',
 57: 'M',
 58: ':',
 59: ')',
 60: 'J',
 61: 'B',
 62: '[',
 63: 'y',
 64: 'r',
 65: ']',
 66: 'p',
 67: '_',
 68: 'c',
 69: 'T',
 70: 'a',
 71: 'w',
 72: 'L',
 73: '?',
 74: 'd',
 75: '8',
 76: '7',
 77: 'G',
 78: 'f',
 79: '!',
 80: 's',
 81: '|',
 82: 'R',
 83: '"'}

In [6]:
# Create an encoder that reads the character and returns the ID.
encoder = {char:ind for ind,char in decoder.items()}
encoder

{'<': 0,
 'z': 1,
 't': 2,
 'b': 3,
 'W': 4,
 '`': 5,
 'e': 6,
 '(': 7,
 'v': 8,
 'o': 9,
 '}': 10,
 'P': 11,
 'n': 12,
 '2': 13,
 'l': 14,
 'N': 15,
 '4': 16,
 'q': 17,
 '9': 18,
 'k': 19,
 'g': 20,
 'E': 21,
 'H': 22,
 'F': 23,
 '1': 24,
 '6': 25,
 'u': 26,
 'X': 27,
 'Q': 28,
 '0': 29,
 '>': 30,
 'S': 31,
 '-': 32,
 'D': 33,
 '\n': 34,
 ';': 35,
 "'": 36,
 '5': 37,
 'U': 38,
 'V': 39,
 ' ': 40,
 'I': 41,
 '.': 42,
 'Y': 43,
 'x': 44,
 ',': 45,
 'j': 46,
 'K': 47,
 '&': 48,
 'C': 49,
 'A': 50,
 'm': 51,
 'h': 52,
 'O': 53,
 'i': 54,
 'Z': 55,
 '3': 56,
 'M': 57,
 ':': 58,
 ')': 59,
 'J': 60,
 'B': 61,
 '[': 62,
 'y': 63,
 'r': 64,
 ']': 65,
 'p': 66,
 '_': 67,
 'c': 68,
 'T': 69,
 'a': 70,
 'w': 71,
 'L': 72,
 '?': 73,
 'd': 74,
 '8': 75,
 '7': 76,
 'G': 77,
 'f': 78,
 '!': 79,
 's': 80,
 '|': 81,
 'R': 82,
 '"': 83}

In [7]:
# Encode the text as a numpy array
encoded_text = np.array([encoder[char] for char in text])
encoded_text[:100]

array([34, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 24, 34, 40, 40, 23, 64,  9, 51, 40, 78, 70, 54,
       64,  6, 80,  2, 40, 68, 64,  6, 70,  2, 26, 64,  6, 80, 40, 71,  6,
       40, 74,  6, 80, 54, 64,  6, 40, 54, 12, 68, 64,  6, 70, 80,  6, 45,
       34, 40, 40, 69, 52, 70,  2, 40,  2, 52,  6, 64,  6,  3, 63, 40,  3,
        6, 70, 26,  2, 63, 36, 80, 40, 64,  9, 80,  6, 40, 51, 54])

## One-hot encoding

We will be creating a one-hot encoding matrix of all characters in the text.

In [9]:
def one_hot_encoder(encoded_text, num_uni_chars):
    '''
    Returns a one-hot encoded matrix of shape (encoded_text.size, unique_characters)
    Parameters
    ----------
    - encoded_text [np.array]: batch of encoded text
    - num_uni_chars [int]: number of unique characters in the text
    '''
    
    # Create a matrix of zeros
    one_hot = np.zeros((encoded_text.size,num_uni_chars))
    
    # Convert the matrix to Float32 to ensure Torch compatibility
    one_hot = one_hot.astype(np.float32)
    
    # One-hot encode original matrix
    one_hot[np.arange(one_hot.shape[0]),encoded_text.flatten()] = 1.0
    
    # Reshape to match the batch size. 
    one_hot = one_hot.reshape((*encoded_text.shape,num_uni_chars))
    
    return one_hot
    

In [13]:
# Sample
one_hot_encoder(np.array([0,2,2,3,1]),4)

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)

## Training batches

The training batches target data will be the data shifted by one position. Instead of providing only the next letter, the entire context will be provided. This will allow the network to learn the gramatical rules, not just the most probable letter.

In [20]:
# Sample
t = [c for c in 'Hello there']
print(f'X = {t[:-1]}')
print(f'y = {t[1:]}')

X = ['H', 'e', 'l', 'l', 'o', ' ', 't', 'h', 'e', 'r']
y = ['e', 'l', 'l', 'o', ' ', 't', 'h', 'e', 'r', 'e']


We need to create a batch generator that will reshape the data to be of shape (batches, elements per batch)

In [49]:
# Example
sample_text = np.arange(100)
print(f'Original text = {sample_text}')
print(f'\nTransformed to 5 batches = \n{sample_text.reshape(10,-1)}')

Original text = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]

Transformed to 5 batches = 
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


In [ ]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    '''
    Generator object to create training batches as requested.
    Parameters
    ----------
    - encoded_text [np.array]: encoded text to be batched
    - samp_per_batch [int]: samples per batch that will be created
    - seq_len [int]: number of characters to include in each sample
    
    Output
    -------
    X [np.array]: encoded text of length seq_len
    y [np.array]: X shifted by one position to the right
    '''
    
    # Calculate total number of characters per batch
    chars_per_batch = samp_per_batch * seq_len
    
    # Calculate the total number of batches that can be made
    num_batches = int(len(encoded_text)/chars_per_batch)
    
    # Remove extra characters that won't fit into a batch
    encoded_text = encoded_text[:num_batches*chars_per_batch]
    
    # Reshape encoded text
    encoded_text.reshape((samp_per_batch,-1))
    
    # Generate sequences
    for i in range(0,encoded_text.shape[1],seq_len):
        
        X = encoded_text[:,i:i+seq_len]
        y = np.zeros_like(x)
        
        # [1,2,3]        
        # [0,0,0] --> [2,3,4]
        
        y[:,:-1] = x[:,1:] # Insert in y the x values shifted by one place. X is one position smaller than y
        y[:,-1] = encoded_text[:i+seq_len] # Insert the following value. This is different to i:i+seq_len as 